In [3]:
import geopandas as gpd
import pandas as pd

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#### BNG to WGS84

In [5]:
from shapely.geometry import Point, Polygon, MultiPoint, MultiLineString
from functools import partial
import pyproj
from shapely.ops import transform

def swap_xy(geom):
    """Swaps the x & y coordinates of a geometry column"""
    
    if geom.is_empty:
        return geom

    if geom.has_z:
        def swap_xy_coords(coords):
            for x, y, z in coords:
                yield (y, x, z)
    else:
        def swap_xy_coords(coords):
            for x, y in coords:
                yield (y, x)

    # Process coordinates from each supported geometry type
    if geom.type in ('Point', 'LineString', 'LinearRing'):
        return type(geom)(list(swap_xy_coords(geom.coords)))
    
    elif geom.type == 'Polygon':
        ring = geom.exterior
        shell = type(ring)(list(swap_xy_coords(ring.coords)))
        holes = list(geom.interiors)
        for pos, ring in enumerate(holes):
            holes[pos] = type(ring)(list(swap_xy_coords(ring.coords)))
        return type(geom)(shell, holes)
    
    elif geom.type.startswith('Multi') or geom.type == 'GeometryCollection':
        # Recursive call
        return type(geom)([swap_xy(part) for part in geom.geoms])
    
    else:
        raise ValueError('Type %r not recognized' % geom.type)

def transform_to_lon_lat(row):
    
    project = partial(
        pyproj.transform,
        pyproj.Proj('epsg:27700'), 
        pyproj.Proj('epsg:4326')
    ) 
    
    return transform(project, row['geometry'])

#### Performing the conversion

In [6]:
gdf = gpd.read_file('zip://./maps/source_maps/UK_LA.zip')
gdf['geometry'] = gdf.apply(transform_to_lon_lat, axis=1)
gdf['geometry'] = gdf['geometry'].map(swap_xy)
gdf.to_file("./maps/hq_maps/UK_LA_LL.json", driver='GeoJSON')
gdf.plot()